이 자료는 위키독스 '딥 러닝을 이용한 자연어 처리 입문'의 FastText 튜토리얼 자료입니다. 해당 코드에 대한 설명은 오프라인 출판물이나 강의에서 확인할 수 있습니다.

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 11.3 MB/s 
     |████████████████████████████████| 448 kB 79.3 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-12-13 04:52:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192

hgtk : https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글 자모 단위 처리 패키지 설치
pip install hgtk

  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6689 sha256=150fed01908c655ecb0fd51cfd2ecf84805c59020aff60e2abc3d5c3f0ce5139
  Stored in directory: /root/.cache/pip/wheels/a6/e9/bc/524beb5222b11aa439a23a07be5bd8a559d266153103c37979
Successfully built hgtk


In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 23.59 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
/content/Mecab-ko-for-Google-Colab/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -m

# 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7f725735e910>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 2. hgtk 튜토리얼

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

In [ ]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [ ]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [ ]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [ ]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

# 3. 데이터 전처리

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char
    
  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [ ]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [ ]:
mecab = Mecab()

In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [00:59<00:00, 3374.62it/s]


In [ ]:
len(tokenized_data)

200000

In [ ]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  
  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']
  
  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:  
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

In [ ]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

# 4. FastText

In [ ]:
import fasttext

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 441194.06 line/s]


In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([-0.39872342,  0.69570994, -0.08290315, -0.0033732 , -0.26300344,
       -1.0410938 ,  0.07197974,  0.05049018,  0.41133216,  0.13120236,
       -0.17054978, -0.7641488 ,  0.82944995,  0.8608577 , -0.25225928,
        0.4545055 ,  0.11331062,  0.3853149 ,  0.23782808,  0.6395339 ,
        0.4978354 ,  0.6740814 , -0.19261919, -0.65290815,  0.2476723 ,
        0.7371382 , -0.07621936, -0.33663422, -0.68590087,  0.3110573 ,
        0.26337826,  0.2677149 ,  0.4448428 , -0.0934477 ,  0.27365035,
       -0.34715688,  0.8025407 ,  0.71842116,  0.38159037,  0.8544624 ,
       -0.32675672, -0.24738057, -0.29285187, -0.5968566 ,  0.4094504 ,
        0.00431124, -0.28558907, -0.8029684 ,  0.04944834,  0.3095079 ,
        1.4729898 , -0.49228516, -1.2144272 ,  0.16817385, -0.3032574 ,
       -0.0485586 , -0.43313843, -0.4085541 , -0.21847768, -1.0622363 ,
        0.3746751 ,  0.37288627,  1.7106302 ,  0.02357475, -0.23483245,
        0.5651234 , -0.5941423 ,  0.03626321,  0.14119053,  0.30

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8772189021110535, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.855356752872467, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7699680328369141, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7600423693656921, 'ㅊㅣㄴㄱㅜ-'),
 (0.7499511241912842, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7111473679542542, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.7054380774497986, 'ㄴㅏㅁㅇㅏ-'),
 (0.7045841217041016, 'ㅈㅗ-ㅋㅏ-'),
 (0.7039702534675598, 'ㅎㅏㄱㅅㅐㅇ'),
 (0.6789932250976562, 'ㅇㅓㄴㄴㅣ-')]

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8772189021110535), ('남친', 0.855356752872467), ('남편', 0.7699680328369141), ('친구', 0.7600423693656921), ('생일', 0.7499511241912842), ('난생', 0.7111473679542542), ('남아', 0.7054380774497986), ('조카', 0.7045841217041016), ('학생', 0.7039702534675598), ('언니', 0.6789932250976562)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))

[('남동생', 0.8929116725921631), ('남친', 0.8108700513839722), ('남매', 0.7864927649497986), ('남짓', 0.7611492872238159), ('남긴', 0.7403371334075928), ('남녀', 0.7286942005157471), ('남김', 0.7269377708435059), ('남겼', 0.724404513835907), ('남아', 0.7095820903778076), ('남길', 0.6867814660072327)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))

[('남동생', 0.8261916041374207), ('남친', 0.7372435331344604), ('남녀', 0.6810236573219299), ('남짓', 0.6739186644554138), ('남김', 0.6714270114898682), ('남매', 0.6663112044334412), ('남긴', 0.6661489605903625), ('남아', 0.6501202583312988), ('남길', 0.6458204388618469), ('남여', 0.6337192058563232)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))

[('남동생', 0.865268349647522), ('난생', 0.8423358798027039), ('남편', 0.7973870038986206), ('남친', 0.7897872924804688), ('동생', 0.7857871055603027), ('남아', 0.7442117929458618), ('중학생', 0.6965839266777039), ('학생', 0.6905945539474487), ('고등학생', 0.6838200092315674), ('나눴', 0.6709653735160828)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.9241121411323547), ('동생', 0.8893769383430481), ('남친', 0.7897419929504395), ('남편', 0.7810367941856384), ('난생', 0.7431588172912598), ('친구', 0.7360448241233826), ('중학생', 0.7242789268493652), ('학생', 0.7208242416381836), ('조카', 0.7056918144226074), ('남아', 0.6924488544464111)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))

[('남동생', 0.9052563309669495), ('동생', 0.8540001511573792), ('남친', 0.7891114354133606), ('남편', 0.7881366014480591), ('난생', 0.7722191214561462), ('중학생', 0.7317784428596497), ('학생', 0.7308444380760193), ('고등학생', 0.7282205820083618), ('초등학생', 0.7080841660499573), ('친구', 0.7018522620201111)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('냚동생'), k=10)))

[('동생', 0.9666105508804321), ('남동생', 0.8954262733459473), ('친구', 0.8230040073394775), ('학생', 0.7806841731071472), ('조카', 0.7685918807983398), ('언니', 0.7610039710998535), ('생일', 0.7600469589233398), ('중학생', 0.7520121932029724), ('초등학생', 0.7360523343086243), ('고등학생', 0.7285748720169067)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품질'), k=10)))

[('품질', 0.8515394926071167), ('음질', 0.8296878337860107), ('땜질', 0.7235048413276672), ('찜질', 0.7046321034431458), ('퀄리티', 0.6999291777610779), ('성질', 0.6669513583183289), ('사포질', 0.6603908538818359), ('고감', 0.6568045020103455), ('군것질', 0.6555950045585632), ('바느질', 0.6496801376342773)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품쥘'), k=10)))

[('고품질', 0.8546069264411926), ('재고품', 0.7780944108963013), ('반제품', 0.7375020980834961), ('중고품', 0.7333138585090637), ('고퀄', 0.7321582436561584), ('재품', 0.7282046675682068), ('제품', 0.725265383720398), ('소모품', 0.7111107707023621), ('소지품', 0.6930672526359558), ('비품', 0.6823447346687317)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('노품질'), k=10)))

[('고품질', 0.8981618285179138), ('품질', 0.8768537044525146), ('음질', 0.7608702182769775), ('퀄리티', 0.7084047198295593), ('땜질', 0.6972792148590088), ('화질', 0.6597926616668701), ('찜질', 0.6569085121154785), ('퀄러티', 0.650342583656311), ('비품', 0.6333506107330322), ('성질', 0.6308069229125977)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('보품질'), k=10)))

[('고품질', 0.8368117809295654), ('품질', 0.7878326177597046), ('음질', 0.7400999665260315), ('재질', 0.7136402130126953), ('퀄리티', 0.7131840586662292), ('보풀', 0.6958552002906799), ('땜질', 0.6932253241539001), ('찜질', 0.666188657283783), ('사포질', 0.6642879247665405), ('화질', 0.6614761352539062)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품'), k=10)))

[('반제품', 0.8867008686065674), ('완제품', 0.8767348527908325), ('상품', 0.8559508323669434), ('타제품', 0.837952733039856), ('재품', 0.8377143740653992), ('제풍', 0.7995226979255676), ('중품', 0.7986012697219849), ('화학제품', 0.79449862241745), ('최상품', 0.7938100695610046), ('명품', 0.7828798890113831)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품ㅋ'), k=10)))

[('제품', 0.8569557666778564), ('제풍', 0.7561682462692261), ('완제품', 0.7530835270881653), ('최상품', 0.716444730758667), ('반제품', 0.7133628726005554), ('상품', 0.7120581865310669), ('타제품', 0.6975999474525452), ('재품', 0.6618362069129944), ('화학제품', 0.6580104231834412), ('상품명', 0.6535097360610962)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품^^'), k=10)))

[('제품', 0.940127432346344), ('제풍', 0.8229137063026428), ('완제품', 0.7972779273986816), ('반제품', 0.7952364087104797), ('타제품', 0.7728006839752197), ('상품', 0.7710467576980591), ('최상품', 0.746713399887085), ('재품', 0.734859049320221), ('화학제품', 0.7087850570678711), ('상품명', 0.689889669418335)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제푼ㅋ'), k=10)))

[('제풍', 0.6965344548225403), ('제품', 0.6107862591743469), ('젝', 0.5349202156066895), ('제왕', 0.5345032811164856), ('젱', 0.5201618075370789), ('최상품', 0.5183104276657104), ('제습', 0.5139129757881165), ('완제품', 0.5121307373046875), ('제약', 0.508080780506134), ('최저', 0.4962596595287323)]
